# Libraries and Mounting

In [ ]:
from google.colab import drive
import pandas as pd
import chess.pgn
import chess
import chess.engine
import pprint
from datetime import datetime

In [ ]:
!pip install python-chess==0.28.3 #required For SimpleEninge for Board evaulation

In [ ]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
cd '/content/drive/Shared drives/DSF_Project'

/content/drive/Shared drives/DSF_Project


# StockFish Engine

In [ ]:
#If Engine has not been compiled

# cd '/content/drive/Shared drives/DSF_Project/StockFish/stockfish-10-linux/src'

In [ ]:
#If Engine has not been compiled

# !make build ARCH=x86-64 

In [ ]:
#If Engine has not been compiled

# cd '/content/drive/Shared drives/DSF_Project'

In [ ]:
#To give the permession 

!chmod +x StockFish/stockfish-10-linux/Linux/stockfish_10_x64

# PGN to CSV

## Functions

### Board Evaluation

In [ ]:
def board_eval(board):
  engine = chess.engine.SimpleEngine.popen_uci("StockFish/stockfish-10-linux/Linux/stockfish_10_x64")
  info = engine.analyse(board, chess.engine.Limit(time=0.2))
  score=info["score"]
  
  engine.quit()
  return score

### Get First Two Moves For White and Black

In [ ]:
def get_first_moves(moves):
  white1=""
  black1=""
  white2=""
  black2=""
  moves_split=moves
  moves_split=moves_split.split(',')
  if(len(moves_split)>=4):
    white1 = moves_split[0]
    black1 = white1 + "," + moves_split[1]
    white2 = black1 + "," + moves_split[2]
    black2 = white2 + "," + moves_split[3]
  else:
    white1 = moves_split[0] if len(moves_split)>=1 else 'NaN'
    black1 = (white1 + "," + moves_split[1]) if len(moves_split)>=2 else 'NaN'
    white2 = (black1 + "," + moves_split[2]) if len(moves_split)>=3 else 'NaN'
    black2 = (white2 + "," + moves_split[3]) if len(moves_split)>=4 else 'NaN'

  return white1,black1,white2,black2

### Get Average Game Score

In [ ]:
def get_game_score(game_eval):
  early=int((25*len(game_eval))/100)
  middle=int((75*len(game_eval))/100)
 
  early_game_eval=0
  middle_game_eval=0
  end_game_eval=0

  for i in range(early):
    early_game_eval=early_game_eval+game_eval[i]

  for i in range(early,middle):
    middle_game_eval=middle_game_eval+game_eval[i]
  
  for i in range(middle,len(game_eval)):
    end_game_eval=end_game_eval+game_eval[i]
  
  
  early_game_eval=early_game_eval/early
  middle_game_eval=middle_game_eval/(middle-early)
  end_game_eval=end_game_eval/(len(game_eval)-middle)

  return early_game_eval,middle_game_eval,end_game_eval

### Features From Moves and Board

In [ ]:
def get_features_from_moves_and_board(game_moves):
  
  moves=""
  clocks=""
  sans=""
  move_count=0
  white_move_count=0
  black_move_count=0
  white_eval=0
  black_eval=0
  white_pawn_promote=0
  black_pawn_promote=0
  white_king_castling=False
  black_king_castling=False
  white_queen_castling=False
  black_queen_castling=False
  white_capture=0
  black_capture=0
  white_checks=0
  black_checks=0
  game_eval=[]
  white_start_time=0
  black_start_time=0
  white_end_time=0
  black_end_time=0
  ref_time=datetime.strptime('0:00:00', '%H:%M:%S')
  white_play_time='NaN'
  black_play_time='NaN'
  white_time_remaining='NaN'
  black_time_remaining='NaN'
  white_eval=0
  black_eval=0
  early_game_eval,middle_game_eval,end_game_eval=0,0,0

  white_move_type={1:0,2:0,3:0,4:0,5:0,6:0}
  black_move_type={1:0,2:0,3:0,4:0,5:0,6:0}

  #Begin Loop
  while game_moves.variations:
      
    #Get Clock      
    clock_comment=game_moves.variation(0).comment

    clock=clock_comment.split(' ')
    if(len(clock)>1 and ('clk' in clock_comment)):
      if 'eval' in clock_comment:
        clock=clock[3][:-1]
      else:
        clock=clock[1][:-1]

      clocks=clocks+clock+","

    #Get SAN
    san=game_moves.variation(0).san()
    sans=sans+san+","

    #Get nag
    nag=game_moves.variation(0).nags

    #Get Moves
    move=game_moves.variation(0).move      
    moves=moves+(str(move))+","

    #Get Board Eval for White and Black
    board=game_moves.board()
    score=board_eval(board)
    
    if(score.is_mate()):
      score=str(score)
      score=score[1:]
      score=int(score)
    else:
      score=int(str(score))

    if board.turn:
      white_eval=white_eval+score #Get White Eval
      white_move_count +=1  #Get White Number of Moves
      white_end_time=clock
      if move.promotion:
        white_pawn_promote+=1 #Get White Pawn Promotions
      if board.is_kingside_castling(move):
        white_king_castling=True  #Get White King Castling
      if board.is_queenside_castling(move):
        white_queen_castling=True #Get White Queen Castling
      if board.is_capture(move):
        white_capture+=1 #Get White Captures
      if board.is_check():
        white_checks+=1 #Get White Checks
      if nag:
        move_type=list(nag)[0]
        white_move_type[move_type] +=1 #Get Types of Move For White --- Brilliant/Blunder/etc
      if move_count==0:
         white_start_time=clock
         move_count+=1       
    
    else:
      black_eval=black_eval+score #Get Black Eval
      black_move_count +=1  #Get Black Number Moves
      black_end_time=clock
      if move.promotion:
        black_pawn_promote+=1 #Get Black Pawn Promotions
      if board.is_kingside_castling(move):
        black_king_castling=True #Get Black King Castling
      if board.is_queenside_castling(move):
        black_queen_castling=True #Get Black Queen Castling
      if board.is_capture(move):
        black_capture+=1 #Get Black Captures
      if board.is_check():
        black_checks+=1 #Get Black Checks
      if nag:
        move_type=list(nag)[0]
        black_move_type[move_type] +=1 #Get Types of Move For Black --- Brilliant/Blunder/etc
      if move_count==1:
         black_start_time=clock

    game_eval.append(score) #For Game Score


    game_moves=game_moves.variation(0)
    
  # End of Loop
  

  #Total Number of Moves
  move_count = len(game_eval)
  
  if(move_count>=4):                                                #Significant only for moves more than 3
    if 'clk' in clock_comment:                                                
      #White Play Time
      white_start_time=datetime.strptime(white_start_time, '%H:%M:%S')
      white_end_time=datetime.strptime(white_end_time, '%H:%M:%S')
      white_play_time=white_start_time-white_end_time
      white_play_time=white_play_time.total_seconds()

      
      #Black Play Time
      black_start_time=datetime.strptime(black_start_time, '%H:%M:%S')
      black_end_time=datetime.strptime(black_end_time, '%H:%M:%S')
      black_play_time=black_start_time-black_end_time
      black_play_time=black_play_time.total_seconds()

      #White Time Remaining
      white_time_remaining=(white_end_time-ref_time).total_seconds()
      
      #Black Time Remaining
      black_time_remaining=(black_end_time-ref_time).total_seconds()

    #Get Average Eval
    white_eval=white_eval/(white_move_count)
    black_eval=white_eval/(black_move_count)

    #Get Average Score at Three different stages
    early_game_eval,middle_game_eval,end_game_eval = get_game_score(game_eval)

  
  #Get Moves,Sans and Clock
  moves = moves[:-1]
  clocks = clocks[:-1]
  sans = sans[:-1]

  #Get Final Board in FEN
  end_board_fen=game_moves.board().fen()

  result={"moves":moves,
          "total_moves":move_count,
          "white_eval":white_eval,
          "black_eval":black_eval,
          "white_moves":white_move_count,
          "black_moves":black_move_count,
          "clocks":clocks,
          "sans":sans,
          "end_board_fen":end_board_fen,
          "early_game_eval":early_game_eval,
          "middle_game_eval":middle_game_eval,
          "end_game_eval":end_game_eval,
          "black_pawn_promote":black_pawn_promote,
          "black_king_castling":black_king_castling,
          "black_queen_castling":black_queen_castling,
          "black_capture":black_capture,
          "black_checks":black_checks,
          "white_pawn_promote":white_pawn_promote,
          "white_king_castling":white_king_castling,
          "white_queen_castling":white_queen_castling,
          "white_capture":white_capture,
          "white_checks":white_checks,
          "black_GOOD_MOVE":black_move_type[1],
          "black_MISTAKE":black_move_type[2],
          "black_BRILLIANT_MOVE":black_move_type[3],
          "black_BLUNDER":black_move_type[4],
          "black_SPECULATIVE_MOVE":black_move_type[5],
          "black_DUBIOUS_MOVE":black_move_type[6],
          "white_GOOD_MOVE":white_move_type[1],
          "white_MISTAKE":white_move_type[2],
          "white_BRILLIANT_MOVE":white_move_type[3],
          "white_BLUNDER":white_move_type[4],
          "white_SPECULATIVE_MOVE":white_move_type[5],
          "white_DUBIOUS_MOVE":white_move_type[6],
          "white_play_time":white_play_time,
          "black_play_time":black_play_time,
          "white_time_remaining":white_time_remaining,
          "black_time_remaining":black_time_remaining
          }

  return result

### Get Start Game

In [ ]:
def get_start_game(start,pgn):
  for i in range(start):
    game=chess.pgn.read_game(pgn)
  
  return i,game

## CSV Extraction

In [ ]:
year='2017'
month='09'
start=5001
end=6000
file_path_PGN="dataset/Standard/PGN/"+year+"/lichess_db_standard_rated_"+year+"-"+month
file_path_CSV="dataset/Standard/CSV/"+year+"/lichess_db_standard_rated_"+year+"-"+month

In [ ]:
%%time
import numpy as np
pgn = open(file_path_PGN+".pgn")

k,game=get_start_game(start,pgn)
# game=chess.pgn.read_game(pgn)
i=0
cols=[]
cols_game=[]
while game!=None and k<end:
  #Get All features based on Moves and Board
  result=get_features_from_moves_and_board(game)

  #Get First Moves of White and Black
  white1,black1,white2,black2=get_first_moves(result['moves'])

  if i==0:
    for col in game.headers:
      cols.append(col)
    cols = ['ID'] + cols
    cols_game=cols.copy()

  row=[game.headers[x] if x in game.headers else "NaN" for x in cols_game]
  row[0]=k;
  
  if(i==0):
    for key in result:
      cols.append(key)
    df=pd.DataFrame(columns=cols)    
  
  for key,value in result.items():
    row.append(value)

  event=row[1].split(' ')
  row[1]=event[1]
  df.loc[i]=row
  i +=1
  k +=1
  # print(i)
  # print("Game Count: ",k)
  # pprint.pprint(result)

  game=chess.pgn.read_game(pgn)

CPU times: user 25min 7s, sys: 6min 10s, total: 31min 18s
Wall time: 9h 50min 44s


In [ ]:
df.to_csv(file_path_CSV+"_PGN_to_CSV_"+str(start)+"_to_"+str(end)+".csv",index=False)

# Board Eval

In [ ]:
# def getBoardScore(board):
#   engine = chess.engine.SimpleEngine.popen_uci("StockFish/stockfish-10-linux/Linux/stockfish_10_x64")
#   info = engine.analyse(board, chess.engine.Limit(time=0.5))
#   score=info["score"]
#   print(info)
#   # print(score.turn)

#   # info = engine.analyse(board, chess.engine.Limit(depth=40))
#   # score_depth=info["score"]
  
#   engine.quit()
#   return score

In [ ]:
# %%time
# pgn = open(file_path_PGN+".pgn")
# # print(file_path_PGN)

In [ ]:
# game=chess.pgn.read_game(pgn)
# print(game)

In [ ]:
# game2=game

In [ ]:
# i=0
# nag_t=0
# # game=chess.pgn.read_game(pgn)
# # game=game2
# test={1:0,2:0,3:0,4:0,5:0,6:0}
# clocks=""

# print(game)
# while game.variations:
#   # print(game_moves.move)
#   board=game.board()

#   nag=game.variation(0).nags
#   # print(board.turn)
#   # print(i+1)
#   # print("Score:",str(score))
#   # print(game.comment)
#   # print(i)
#   # print(score.is_mate())
#   # print(i+int(str(score)))
#   # if nag:
#   #     print(nag)
#   #     move_type=list(nag)[0]
#   #     print(move_type)
#   #     test[move_type] +=1 #Get Types of Move For Black --- Brilliant/Blunder/etc
#   # eval_present=False
#   clock_comment=game.variation(0).comment
#   # clock_comment=""
#   clock=clock_comment.split(' ')
#   print("sd",clock)
#   if(len(clock)>1):
#     if 'eval' in clock_comment:
#       clock=clock[3][:-1]
#     else:
#       clock=clock[1][:-1]
  
#   print(type(clock))
#   clocks=clocks+clock+","
#   # print(clocks)

#   # if nag:
#   #   print(nag)
#   #   comment=game.variation(0).comment
#   #   print(comment)
#   #   nag_t=nag


#   # print(sans)
#   # comment=game.variation(0).comment
#   # print(comment)
#   # if move.promotion:
#   #   print(sans)
#   #   print(move)
#   #   print(move.promotion)

#   # if board.is_check():
#   #   print(board.turn)
#   #   print(sans)
#   #   print("Captured\n")5
#   # print(nags)
#   game=game.variation(0)
#   i=i+1

In [ ]:
# print(test)
# for key in test:
#   print(key)

In [ ]:
# from datetime import datetime

# test='0:01:00'
# test2='0:00:59'
# date = datetime.strptime(test, '%H:%M:%S')
# date2 = datetime.strptime(test2, '%H:%M:%S')
# # date=date.time().isoformat()
# # date2=date2.time().isoformat()
# ref_time=datetime.strptime('0:00:00', '%H:%M:%S')
# print(ref_time)
# print(date)
# a=date-ref_time
# print((date-ref_time).total_seconds())
# print(date.second)
# # print(type(date2))
# ans=date-date2
# # a=datetime.timedelta(hours=ans.tm_hour,minutes=ans.tm_min,seconds=ans.tm_sec).total_seconds()
# print(type(ans))
# print(type(ans.total_seconds()))

In [ ]:
# t=list(nag_t)[0]
# print(t)
# # print(t[0])
# k=t
# d={6:0}


In [ ]:

# d[k]+=1
# print(d[k])

# Testing

In [ ]:
# year='2019'
# month='10'
# limit=2
# file_path_PGN="dataset/Standard/PGN/"+year+"/lichess_db_standard_rated_"+year+"-"+month
# file_path_CSV="dataset/Standard/CSV/"+year+"/lichess_db_standard_rated_"+year+"-"+month

In [ ]:
# %%time
# pgn = open(file_path_PGN+".pgn")

# game=chess.pgn.read_game(pgn)
# i=0
# leng=[]
# nag_c=[]
# while game!=None:
#   game_moves=game
#   j=0
#   while game_moves.variations:
#     nag=game_moves.variation(0).nags
#     game_moves=game_moves.variation(0)
#     if nag:
#       nag_c.append(nag)
#       print(nag)
#       print(i)

#     j+=1
#   i=i+1
#   leng.append(i)
#   game=chess.pgn.read_game(pgn)
